In [1]:
!pip install --upgrade selenium
!pip install --upgrade webdriver_manager
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import re
import time
from selenium.webdriver.common.by import By
import requests
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
#request user input for LinkedIn username and password:
print("Please enter the exact LinkedIn username you use to login (email/phone?):")
username_string = str(input())
print()
print("Please enter the exact LinkedIn password:")
password_string = str(input())
print()
print("Please enter your usernmae exactly how it appears in your profile link (after '/in') :")
link_username = str(input())
print()


#create a browser-specific (Google Chrome) web navigation simulator:
service = Service()
options = webdriver.ChromeOptions()
browser = webdriver.Chrome(service=service, options=options)

#open the LinkedIn login page and login under a specified account:
browser.get('https://www.linkedin.com/login')
#enter the specified information to login to LinkedIn:
elementID = browser.find_element(By.ID,'username')
elementID.send_keys(username_string)
elementID = browser.find_element(By.ID,'password')
elementID.send_keys(password_string)
elementID.submit()


urls = ["https://www.linkedin.com/posts/chirag-bajaj-318705117_im-happy-to-share-that-im-starting-a-new-activity-7103316878118297600-m_Eo?utm_source=share&utm_medium=member_desktop"]

likers=[]
Url=[]
comments=[]
C_Url=[]

for url in urls:
    browser.get(url)
    time.sleep(1)
    try:
        click = browser.find_element(By.CLASS_NAME, "social-details-social-counts__count-value")
        browser.execute_script("arguments[0].click();", click)
        while True:
            people = browser.find_elements(By.XPATH, "//ul[@class='artdeco-list artdeco-list--offset-1']/li")
            if not people:
                print('List "people" is empty')
                break
            show_more_btn = browser.find_elements(By.XPATH, '//div[@id="artdeco-modal-outlet"]//button[contains(@class,"load-button")]')
            if show_more_btn:
                try:
                    show_more_btn[0].click()
                except StaleElementReferenceException:
                    # Refresh the list of elements
                    show_more_btn = browser.find_elements(By.XPATH, '//div[@id="artdeco-modal-outlet"]//button[contains(@class,"load-button")]')
                    continue
            for person in people:
                try:
                    name_element = person.find_element(By.XPATH, './/div[contains(@class,"artdeco-entity-lockup__title")]')
                    name = name_element.text.split('\n')[0]
                    url_element = person.find_element(By.XPATH, './/div[@class="inline-flex full-width"]/a')
                    url = url_element.get_attribute('href')
                    likers.append(name)
                    Url.append(url)
                except NoSuchElementException:
                    continue  # Skip this person and continue with the next iteration
                finally:
                    # Remove the person element from the DOM
                    browser.execute_script('var element = arguments[0]; element.remove();', person)
                time.sleep(1)
    except NoSuchElementException:
        print("Element not found. Skipping URL:", url)
        continue
    except NoSuchElementException:
        print("Element not found. Skipping URL:", url)
        continue



print(likers)
print(Url)

Please enter the exact LinkedIn username you use to login (email/phone?):
rekha00bajaj@gmail.com

Please enter the exact LinkedIn password:
Rekha00bajaj

Please enter your usernmae exactly how it appears in your profile link (after '/in') :
/chirag-bajaj-318705117/



WebDriverException: ignored

In [ ]:
import pandas as pd
data={'Name_of_Likers':likers,'URL':Url}
df=pd.DataFrame(data,index=range(1,len(likers)+1))
df.to_csv("Likers.csv")
df